In [487]:
import os
import pandas as pd

In [495]:
class LoadExcel:
    
    fileName = None
    columnNames = None
    columnNameRowIndex = 0
    filePath = None
    qualifieldFilePath = None
    sheetNames = None
    
    def __init__(self,filePath=None, fileName = None,columnNames = None , sheetNames=None , columnNameRowIndex = 0):
        self.filePath = filePath
        self.fileName = fileName
        self.columnNames = columnNames
        self.columnNameRowIndex = columnNameRowIndex
        self.sheetNames = sheetNames
        if fileName is not None and filePath is not None:
            self.qualifieldFilePath = str(filePath) +"/"+ str(fileName)
            
    def loadFile(self, readsheet):
        fload = None        
        print("Readin sgeet :"+readsheet)
        if isinstance(readsheet, str) :
            fload = pd.read_excel(self.qualifieldFilePath,sheet_name= readsheet)
        return fload
    
    def get_col_name_frm_row(self, df,rowno, startColNo,endColNo):
        a1 = df.iloc[0,0:11]
        cls = [str(x) for x in a1.values]
        curnt =None
        for i,x in enumerate(cls):
            curnt = x
            if i >0:
                for j,y in enumerate(cls):
                    if j >0 and y==x and i!=j:
                        cls[j] = y+"_"+str(j)
        return cls
    
    def rename_col(self,df, colNameArray):
        xlx1 = df.set_axis(colNameArray, axis=1, inplace=False)
        return xlx1
    
    def get_schema_cols(self,df,colNameArray):
        xlx2 = df[colNameArray]
        return xlx2


class SchemaPrep:
    
    df = None
    cols = None
    schemaNameCol = None
    
    def __init__(self, df=None, cols=None,schemaNameCol=None):
        self.df = df
        self.cols = cols
        self.schemaNameCol = schemaNameCol
        
    def get_unq_schema_names(self,df=None,colNm=None):
        df = df if df is not None else self.df        
        colNm = colNm if colNm is not None else self.schemaNameCol
        rtn = None
        if df is not None and colNm is not None:
            a = df[colNm]
            rtn= [str(x) for x in a.values]            
        return rtn
    
    def create_dot_dicts(self,dct,colAr):
        for x in colAr:
            print()
            
        
    def create_dicts(self,pdct,colAr):        
        for x in colAr:
            if "." not in x:
                pdct[x] = {}
        for key in pdct:
            subs = [y[len(key+"."):] for y in colAr if key+"." in y]
            if len(subs)>0:
                self.create_dicts(pdct[key],subs)
                
    def get_table_objs(self,colAr):  
        dct ={}
        sp.create_dicts(dct,colAr)
        return dct
    
    def json_property_template(self,descrp=None,typ=None):
        dct = {
            "description":"" if descrp is None else descrp.title(),
            "type":"string" if typ is None else typ.lower()
        }
        if typ is None:
            dct["type"] ="string"
        else:
            if typ.lower() == "object":
                dct["properties"]={}
            elif typ.lower() == "array":
                dct["items"]={}
            elif typ.lower() == "ref":
                dct["$ref"]= {}
        return dct
    
    def json_schema_template(self,loc=None,title=None,reg=None, adprop=None,descrp=None):
        dct = {
            "$id": None,
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "title":None,
            "description":None,
            "type":"object",
            "propertyNames":{
                "pattern":None
            },
            "properties":{
                
            },
            "additionalProperties":False if adprop is None else adprop
            
        }
        dct["$id"] = "".join(loc if loc is not None else "https://localhost/schema/{}")
        dct["title"]="".join(title if title is not None else " Json Scheam design for {}")
        dct["description"]="".join(descrp if descrp is not None else " This is Description {}"),
        dct["propertyNames"]["pattern"]= "".join("^[A-Za-z0-9_]+$" if reg is None else reg)
        if adprop is None or adprop == True:
            dct["additionalProperties"]="true"
        else:
            dct["additionalProperties"]="false"
        return dct
        

class GenerateSchema:
    scma = None
    sp = None
    xlx = None
    scDf = None
    inps = {
        "filePath" : "/Users/anand/Documents/sayansi/workspaces/condaws/exceljson",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Source Topic",
         "Schema",
         "Attribute Name",
         "Type",
         "Source To Target Mapping.Transformation Logic",
         "Topc",
         "Schema",
         "Field",
         "Length",
         "Type",
         "Comment"],
        "columnNameRowIndex" : 0,
        "schemaCols" : [
         "Schema_6",
         "Field",
         "Length",
         "Type",
         "Comment"],
        "sheetNames":["Sheet1","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":11,
        "schemaNmsCol":"Schema_6",
        "sheetName":"Sheet1"
    }
    
    def __init__(self,dct=None):
        self.inps = dct if dct is not None else self.inps
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
        self.sp = SchemaPrep()
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet1") 
        
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            self.xlx = xlx2
        else:
            self.xlx = xlx
        return self.xlx
    
    def optaion_schema_frame(self, schemaCols=None):
        # create sub Data Frame for selecting columns which have schema prep columns
        schemaCols = schemaCols if schemaCols is not None else self.inps['schemaCols']
        j1 = self.le.get_schema_cols(self.xlx,schemaCols)
        self.sp.df = j1
        self.scDf = j1
    
    def generate_table_name_object(self,schemaNmsCol=None):
        schemaNmsCol  = schemaNmsCol if schemaNmsCol is not None else self.inps['schemaNmsCol']       
        aa = self.sp.get_unq_schema_names(self.sp.df,schemaNmsCol)[1:]
        dct =self.sp.get_table_objs(aa)
        return dct
    
    def generate_table(self):
        loc="https://localhost/schema/product.schema.json",
        title="Product Final NoSQL JSON Schema",
        reg="^[A-Za-z0-9_]+$", 
        adprop=False,
        descrp="This is Final Product Master Attribute Schema \
                which is hosted for Keeping Data From Final Kafka Product Topic"
        jsn = self.sp.json_schema_template(loc,title,reg,adprop,descrp)
        return jsn
    def generate_table_prperties(self, tables,jsn,mtchToken=None):        
        for key, value in tables.items():
            key1 = key
            if mtchToken is not None:
                key = mtchToken + "." + key
                
            d1 = self.scDf[self.scDf.Schema_6 == key]
            print(type(d1))
            print("org Rows::"+str(self.scDf.size))
            if isinstance(value,dict):
                jsn["properties"][key1] = self.sp.json_property_template(key,"object")
                for row in d1.itertuples():
                    print(row.Field)
                    print(row.Length)
                    print(row.Type)
                    print(row.Comment)
                    jsn["properties"][key1]["properties"][row.Field] = self.sp.json_property_template(row.Comment,row.Type)
                self.generate_table_prperties( value,jsn["properties"][key1],key)
            else:
                jsn["properties"][key1] = self.sp.json_property_template(key1,"string")
        
        
    
    def runfx(self):
        self.loadFle()
        print(self.xlx.size)
        self.xlx.head(2)
        self.optaion_schema_frame()
        tables = self.generate_table_name_object()  
        jsn = self.generate_table()
        self.generate_table_prperties(tables,jsn)
        print(jsn)
        return tables


In [496]:
'''
fl_path = "/Users/anand/Documents/sayansi/workspaces/condaws/exceljson"
fl_name = "kfk.xlsx"
fl_cols = ["Source Topic",
 "Schema",
 "Attribute Name",
 "Type",
 "Source To Target Mapping.Transformation Logic",
 "Topc",
 "Schema",
 "Field",
 "Length",
 "Type",
 "Comment"]
fl_clIdx = 0

fln = fl_path +"/"+ fl_name
#print(le.qualifieldFilePath)
le = LoadExcel(fl_path, fl_name,fl_cols ,fl_clIdx)
xlx = le.loadFile("Sheet1")
clns = le.get_col_name_frm_row(xlx,0,0,11)
xlx2 = le.rename_col(xlx, clns)

fl_cols1 = [
 "Schema_6",
 "Field",
 "Length",
 "Type",
 "Comment"]
j1 = le.get_schema_cols(xlx2,fl_cols1)
sp = SchemaPrep(j1,fl_cols1,"Schema_6")
j1.head(3)
aa = sp.get_unq_schema_names()[1:]
#print(aa)
dct =sp.get_table_objs(aa)
print(dct)
'''

gs = GenerateSchema(inps)
tables = gs.runfx()
#print(tables)


Readin sgeet :Sheet1
143
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARSname
20
int
This Is MARS MARSname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.SUNname
21
string
This Is MARS.SUN MARS.SUNname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.MOONname
22
int
This Is MARS.MOON MARS.MOONname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.EARTHname
23
int
This Is MARS.EARTH MARS.EARTHname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.EARTH.PLANETname
24
string
This Is MARS.EARTH.PLANET MARS.EARTH.PLANETname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.EARTH.SATALITEname
25
int
This Is MARS.EARTH.SATALITE MARS.EARTH.SATALITEname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.SOLARname
26
int
This Is MARS.SOLAR MARS.SOLARname field
<class 'pandas.core.frame.DataFrame'>
org Rows::65
MARS.SOLAR.BLAKHOLEname
27
string
This Is MARS.SOLAR.BLAKHOLE MARS.SOLAR.BLAKHOLEname field
<class 'pandas

In [ ]:
'''
loc="https://localhost/schema/product.schema.json",
title="Product Final NoSQL JSON Schema",
reg="^[A-Za-z0-9_]+$", 
adprop=False,
descrp="This is Final Product Master Attribute Schema \
        which is hosted for Keeping Data From Final Kafka Product Topic"
jsn = gs.sp.json_schema_template(loc,title,reg,adprop,descrp)

for key, value in tables.items(): 
    d1 = gs.scDf[gs.scDf.Schema_6 == key]
    if isinstance(value,dict):
        jsn["properties"][key] = gs.sp.json_property_template(key,"object")
        for row in d1.itertuples():
            print(row.Field)
            print(row.Length)
            print(row.Type)
            print(row.Comment)
            jsn["properties"][key]["properties"][row.Field] = gs.sp.json_property_template(row.Comment,row.Type)
    else:
        jsn["properties"][key] = gs.sp.json_property_template(key,"object")
            
        
        #jsn["properties"][key] = gs.sp.json_property_template()
print(jsn)
'''
#for i in gs.scDf.items():
#    print(i)